In [1]:


import pandas as pd
from datetime import datetime
from nba_api.stats.endpoints import playergamelog
from sqlalchemy.dialects.postgresql import insert


from models import Team
import date_utils as date_mng
from data_manager import DataManager
data_manager = DataManager()
self = data_manager

In [3]:
# def pull all team games
teams = self.query_teams()
test_team = teams[1]


In [49]:
season_type = "Playoffs"
season = "2023-24"
team = test_team

# synchronize games
# session = get_session()
games = self.pull_games_by_team_and_season(team, season, season_type)
game = games.iloc[[0]]

adv_player_stats, adv_team_stats = self.pull_advanced_stats_for_game(game.GAME_ID)
adv_player_stats = adv_player_stats[adv_player_stats["TEAM_ID"] == team.nba_team_id]
adv_team_stats  = adv_team_stats[adv_team_stats["TEAM_ID"] == team.nba_team_id]

player_stats, team_stats = self.pull_traditional_stats_for_game(game.GAME_ID)
print(team_stats)

      GAME_ID     TEAM_ID  TEAM_NAME TEAM_ABBREVIATION  TEAM_CITY  \
0  0042300204  1610612739  Cavaliers               CLE  Cleveland   
1  0042300204  1610612738    Celtics               BOS     Boston   

             MIN  FGM  FGA  FG_PCT  FG3M  ...  OREB  DREB  REB  AST  STL  BLK  \
0  240.000000:00   41   94   0.436    15  ...     8    24   32   26    8    3   
1  240.000000:00   38   78   0.487    12  ...    10    38   48   15    6    4   

   TO  PF  PTS  PLUS_MINUS  
0   7  17  102        -7.0  
1  14  12  109         7.0  

[2 rows x 25 columns]


In [31]:
from models import Game, TeamStats, PlayerStats
season_type = "Playoffs"
season = "2023-24"
team = test_team

def synchronize_games():
    games = self.pull_games_by_team_and_season(team, season, season_type)
    game = games.iloc[[0]]
    pass

def sync_game(game):
    session = get_session()

    game_summary = self.pull_game_summary(game.GAME_ID)
    game = pd.merge(game, game_summary, on="GAME_ID", how="inner")

    insert_statement = insert(Game).values(
        
        nba_game_id = game['GAME_ID'],
        date = game['GAME_DATE'],
        game_status_text = game['GAME_STATUS_TEXT'],
        season = season,
        season_type = season_type,
        home_team_id = team.id if 'vs.' in game['MATCHUP'] else None,
        away_team_id = team.id if '@' in game['MATCHUP'] else None,
        live_period = game['LIVE_PERIOD'],

    )

    update_fields = {
        'date': insert_statement.excluded.date,
        'game_status_text': insert_statement.excluded.game_status_text,
        'season': insert_statement.excluded.season,
        'season_type': insert_statement.excluded.season_type,
        'home_team_id': insert_statement.excluded.home_team_id,
        'away_team_id': insert_statement.excluded.away_team_id,
        'live_period': insert_statement.excluded.live_period,
    }

    upsert_statement = insert_statement.on_conflict_do_update(
        index_elements=['nba_game_id'],
        set_=update_fields
    ).returning(Game.game_id)

    result = session.execute(upsert_statement)
    session.commit() 
    game_id = result.fetchone()[0]

    adv_player_stats, adv_team_stats = self.pull_advanced_stats_for_game(game.GAME_ID)
    adv_player_stats = adv_player_stats[adv_player_stats["TEAM_ID"] == team.nba_team_id]
    adv_team_stats  = adv_team_stats[adv_team_stats["TEAM_ID"] == team.nba_team_id]

    player_stats, team_stats = self.pull_traditional_stats_for_game(game.GAME_ID)
    player_stats = player_stats[player_stats["TEAM_ID"] == team.nba_team_id]
    team_stats = team_stats[team_stats["TEAM_ID"] == team.nba_team_id]

    # create dicts that map nba_ids with local_ids for teams and players
    # then, 
    # create classes if need to, then fill tables for all, like:

    # insert_statement = insert(TeamStats).values(
    #     game_id=game_id,
    #     team_id=team.id,
    #     wl=game['WL'],
    #     points=game['PTS'],
    #     fgm=game['FGM'],
    #     fga=game['FGA'],
    #     fg_pct=game['FG_PCT'],
    #     fg3m=game['FG3M'],
    #     fg3a=game['FG3A'],
    #     fg3_pct=game['FG3_PCT'],
    #     ftm=game['FTM'],
    #     fta=game['FTA'],
    #     ft_pct=game['FT_PCT'],
    #     oreb=game['OREB'],
    #     dreb=game['DREB'],
    #     reb=game['REB'],
    #     ast=game['AST'],
    #     stl=game['STL'],
    #     blk=game['BLK'],
    #     tov=game['TOV'],
    #     pf=game['PF'],
    #     plus_minus=game['PLUS_MINUS']
    # )

    # upsert_statement = insert_statement.on_conflict_do_update(
    #     index_elements=['game_id', 'team_id'],  
    #     set_={
    #         'wl': insert_statement.excluded.wl,
    #         'points': insert_statement.excluded.points,
    #         'fgm': insert_statement.excluded.fgm,
    #         'fga': insert_statement.excluded.fga,
    #         'fg_pct': insert_statement.excluded.fg_pct,
    #         'fg3m': insert_statement.excluded.fg3m,
    #         'fg3a': insert_statement.excluded.fg3a,
    #         'fg3_pct': insert_statement.excluded.fg3_pct,
    #         'ftm': insert_statement.excluded.ftm,
    #         'fta': insert_statement.excluded.fta,
    #         'ft_pct': insert_statement.excluded.ft_pct,
    #         'oreb': insert_statement.excluded.oreb,
    #         'dreb': insert_statement.excluded.dreb,
    #         'reb': insert_statement.excluded.reb,
    #         'ast': insert_statement.excluded.ast,
    #         'stl': insert_statement.excluded.stl,
    #         'blk': insert_statement.excluded.blk,
    #         'tov': insert_statement.excluded.tov,
    #         'pf': insert_statement.excluded.pf,
    #         'plus_minus': insert_statement.excluded.plus_minus
    #     }
    # )

    session.execute(upsert_statement)
    session.commit()
    # sync advanced team stats

    # sync advanced_stats and traditional stats for players (another loop)
    adv_player_stats, adv_team_stats = self.pull_advanced_stats_for_game(game.GAME_ID)
    adv_player_stats = adv_player_stats[adv_player_stats["TEAM_ID"] == team.nba_team_id]
    adv_team_stats  = adv_team_stats[adv_team_stats["TEAM_ID"] == team.nba_team_id]

    player_stats, team_stats = self.pull_traditional_stats_for_game(game.GAME_ID)
    player_stats = player_stats[player_stats["TEAM_ID"] == team.nba_team_id]
    team_stats = team_stats[team_stats["TEAM_ID"] == team.nba_team_id]






  SEASON_ID     TEAM_ID TEAM_ABBREVIATION       TEAM_NAME     GAME_ID  \
0     42023  1610612738               BOS  Boston Celtics  0042300204   

    GAME_DATE    MATCHUP WL  MIN  PTS  ...  GAME_STATUS_TEXT         GAMECODE  \
0  2024-05-13  BOS @ CLE  W  239  109  ...             Final  20240513/BOSCLE   

   HOME_TEAM_ID  VISITOR_TEAM_ID  SEASON  LIVE_PERIOD  LIVE_PC_TIME  \
0    1610612739       1610612738    2023            4                 

   NATL_TV_BROADCASTER_ABBREVIATION  LIVE_PERIOD_TIME_BCAST  WH_STATUS  
0                         TNT/truTV    Q4       - TNT/truTV          1  

[1 rows x 41 columns]


In [5]:
game_summaries = []
for game_summary = self.pull_game_summary(game_id)

In [9]:
print(games.iloc[[0]])
print(game_summary)

  SEASON_ID     TEAM_ID TEAM_ABBREVIATION       TEAM_NAME     GAME_ID  \
0     42023  1610612738               BOS  Boston Celtics  0042300204   

    GAME_DATE    MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  STL  \
0  2024-05-13  BOS @ CLE  W  239  109  ...   0.875    10    38   48   15    6   

   BLK  TOV  PF  PLUS_MINUS  
0    4   14  12         7.0  

[1 rows x 28 columns]
         GAME_DATE_EST  GAME_SEQUENCE     GAME_ID  GAME_STATUS_ID  \
0  2024-05-13T00:00:00              1  0042300204               3   

  GAME_STATUS_TEXT         GAMECODE  HOME_TEAM_ID  VISITOR_TEAM_ID SEASON  \
0            Final  20240513/BOSCLE    1610612739       1610612738   2023   

   LIVE_PERIOD LIVE_PC_TIME NATL_TV_BROADCASTER_ABBREVIATION  \
0            4                                     TNT/truTV   

  LIVE_PERIOD_TIME_BCAST  WH_STATUS  
0   Q4       - TNT/truTV          1  
